<a href="https://colab.research.google.com/github/curiosity806/2020_dacon_satellite_precipitation/blob/deagwon/normalization__augment_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Conv2DTranspose, MaxPooling2D, BatchNormalization, Activation, concatenate, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
import warnings
import gc
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy import stats
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# style 설정
plt.style.use('dark_background')
plt.style.use('seaborn-colorblind')

In [4]:
tf.__version__

'2.2.0'

## 데이터 받아오기

In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)
random.seed(7)
tf.random.set_seed(7)

# 모델만들기

In [0]:
#train_normalization = np.load('/content/drive/My Drive/2020 Kaggle Study/data/part_normalization/part_normalization_onlyrain.npy')
train_normalization = np.load('train_normalization.npy')


index = 0
rain_index = []
for file in train_normalization:
    target= file[:,:,-1].reshape(40,40,1)
    cutoff_labels = np.where(target < 0, 0, target)
    if (cutoff_labels > 0).sum() < 50:
        index = index + 1
        continue
    rain_index.append(index)
    index = index + 1
#train_normalization = train_normalization[rain_index]
#np.save('/content/drive/My Drive/2020 Kaggle Study/data/part_normalization/part_normalization_onlyrain.npy', train_normalization)
#train_normalization.shape

In [0]:
train_normalization = train_normalization[rain_index]

In [0]:
def augment_generator(train):
    rotate = np.zeros([train.shape[0]*8,train.shape[1],train.shape[2],train.shape[3]])
    ch_num = train.shape[3]
    train_len = train.shape[0]
    rotate[:train_len,:,:,:] = train
    del train
    rotate[train_len : train_len * 2,:,:,:] = np.rot90(rotate[:train_len,:,:,:], axes = (1,2))
    rotate[train_len * 2 : train_len * 3,:,:,:] = np.rot90(rotate[train_len : train_len * 2,:,:,:], axes = (1,2))
    rotate[train_len * 3 : train_len * 4,:,:,:] = np.rot90(rotate[train_len * 2 : train_len * 3,:,:,:], axes = (1,2))
    rotate[train_len * 4 :,:,:,:] = np.transpose(rotate[: train_len * 4,:,:,:], axes = (0,1,2,3))
    return rotate

In [0]:
def create_model():
    inputs=Input((40,40,10))
    
    bn=BatchNormalization()(inputs)
    conv0=Conv2D(256, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    bn=BatchNormalization()(conv0)
    conv=Conv2D(128, kernel_size=2, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([conv0, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    conv=Conv2D(64, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
    concat=concatenate([concat, conv], axis=3)
        
    for i in range(5):
        bn=BatchNormalization()(concat)
        conv=Conv2D(32, kernel_size=3, strides=1, padding='same', activation='relu')(bn)
        concat=concatenate([concat, conv], axis=3)
    
    bn=BatchNormalization()(concat)
    outputs=Conv2D(1, kernel_size=1, strides=1, padding='same', activation='relu')(bn)
    
    model=Model(inputs=inputs, outputs=outputs)
    
    return model

In [0]:

#model = Model(input_layer, output_layer)
model = create_model()

In [0]:
from sklearn.metrics import f1_score

def mae(y_true, y_pred) :
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    over_threshold = y_true >= 0.1
    return np.mean(np.abs(y_true[over_threshold] - y_pred[over_threshold]))

def fscore(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true >= 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    return(f1_score(y_true, y_pred))

def maeOverFscore(y_true, y_pred):
    return mae(y_true, y_pred) / (fscore(y_true, y_pred) + 1e-07)

def fscore_keras(y_true, y_pred):
    score = tf.py_function(func=fscore, inp=[y_true, y_pred], Tout=tf.float32, name='fscore_keras')
    return score

def maeOverFscore_keras(y_true, y_pred):
    score = tf.py_function(func=maeOverFscore, inp=[y_true, y_pred], Tout=tf.float32,  name='custom_mse') 
    return score

In [0]:
model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])

## 데이터 제네레이터 만들기

In [0]:
# GMI좌표로 보정된 데이터
gmi_pre = np.load('/content/drive/My Drive/2020 Kaggle Study/data/gmi_preci_32bits.npy')

In [0]:
def augment_generator(train):
    rotate = np.zeros([train.shape[0]*8,train.shape[1],train.shape[2],train.shape[3]])
    ch_num = train.shape[3]
    train_len = train.shape[0]
    rotate[:train_len,:,:,:] = train
    del train
    rotate[train_len : train_len * 2,:,:,:] = np.rot90(rotate[:train_len,:,:,:], axes = (1,2))
    rotate[train_len * 2 : train_len * 3,:,:,:] = np.rot90(rotate[train_len : train_len * 2,:,:,:], axes = (1,2))
    rotate[train_len * 3 : train_len * 4,:,:,:] = np.rot90(rotate[train_len * 2 : train_len * 3,:,:,:], axes = (1,2))
    rotate[train_len * 4 :,:,:,:] = np.transpose(rotate[: train_len * 4,:,:,:], axes = (0,1,2,3))
    return rotate

class MultiOutputDataGenerator(tf.keras.preprocessing.image.ImageDataGenerator):
    def flow(self,
             x,
             y,
             batch_size=None,
             shuffle=True,
             sample_weight=None,
             seed=None,
             save_to_dir=None,
             save_prefix='',
             save_format='png',
             subset=None):

        for flowx, flowy in super().flow(x,y, batch_size= batch_size, shuffle = shuffle):
            flowx = augment_generator(flowx)
            flowy = augment_generator(flowy)
            yield (flowx, flowy)

In [46]:
train_normalization.shape

(44939, 40, 40, 11)

In [0]:
from sklearn.model_selection import KFold
def train_model(x_data, y_data, k, s):
    
    k_fold = KFold(n_splits=k, shuffle=True, random_state=7777)
    
    model_number = 0
    for train_idx, val_idx in k_fold.split(x_data):
        if model_number == s:
            x_train, y_train = x_data[train_idx], y_data[train_idx]
            x_val, y_val = x_data[val_idx], y_data[val_idx]

            # 데이터를 부풀릴시 많은 양의 메모리가 필요
            #x_train, y_train = data_generator(x_train, y_train)

            model = create_model()

            model.compile(loss="mae", optimizer="adam", metrics=[maeOverFscore_keras, fscore_keras])

            callbacks_list = [
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    patience=3,
                    factor=0.8
                ),

                tf.keras.callbacks.ModelCheckpoint(
                    filepath = 'models/model'+str(model_number)+'.h5',
                    monitor='val_score',
                    save_best_only=True
                )
            ]
            datagen = MultiOutputDataGenerator()
            datagen.fit(x_train)

            model.fit(datagen.flow(x_train, y_train, batch_size=16), epochs=50,  validation_data=(x_val, y_val), callbacks=callbacks_list)
        
        model_number+=1

In [0]:
train_normalization[:,:,:,-1].reshape(-1,40,40,1).shape

(30760, 40, 40, 1)

In [0]:
train_normalization.shape

(30760, 40, 40, 11)

In [0]:
from tensorflow.keras.models import load_model, Model
k = 5
models = []

train_model(train_normalization[:,:,:,:-1], train_normalization[:,:,:,-1].reshape(-1,40,40,1), k=k, s=4)

for n in range(k):
    model = load_model('/content/drive/My Drive/2020 Kaggle Study/deagwon/model'+str(n)+'.h5', custom_objects = {'score':score,'fscore_keras':fscore_keras})
    models.append(model)

Epoch 1/50
    431/Unknown - 145s 336ms/step - loss: 4.3946 - maeOverFscore_keras: 3.0396 - fscore_keras: 0.5841

## submission 만들기

In [0]:
dr = [(-1, -1), (-1, 0), (-1, 1),
      (0, -1), (0, 0), (0, 1),
      (1, -1), (1, 0), (1, 1)]

# p1, p2: shape=(-1, 2).
def get_dist(p1, p2):
    x1 = np.deg2rad(p1[:,0])
    y1 = np.deg2rad(p1[:,1])
    x2 = np.deg2rad(p2[:,0])
    y2 = np.deg2rad(p2[:,1])
    dlon = x2 - x1
    dlat = y2 - y1
    a = np.sin(dlat/2)**2 + np.cos(y1) * np.cos(y2) * np.sin(dlon/2)**2 
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))    
    return 6373.0 * c  # km, shape=(-1).

# ori_ll에서의 value 값을 tgt_ll에 대한 값으로 바꾼다.
# value: (40, 40, -1)
# ori_ll, tgt_ll: (40, 40, 2)
# ori_ll: value에 대응되는 원래 latitude, longitude.
# tgt_ll: 변환되는 값에 대응되는 latitude, longitude.
def compen_ll(value, ori_ll, tgt_ll):
    ret = np.empty_like(value)

    n = value.shape[0]
    m = value.shape[1]

    for i in range(n):
        for j in range(m):
            nears = []  # (row, col, value)
            for k in range(9):
                ii = i + dr[k][0]
                jj = j + dr[k][1]
                if ii >= 0 and ii < n and jj >= 0 and jj < m:
                    nears.append((ori_ll[ii, jj][0], ori_ll[ii, jj][1],
                                  tgt_ll[i, j][0], tgt_ll[i, j][1],
                                  value[ii, jj]))

            nears = np.array(nears)  # shape=(-1, 5)
            dists = get_dist(nears[:, 0:2], nears[:, 2:4]).reshape(-1, 1)
            values = nears[:, 4].reshape(-1, 1)
            nears = np.concatenate((dists, values), 1)
            nears = nears[np.argsort(nears[:, 0])]  # sort by dist

            weights = 1 / (nears[:, 0] ** 2 + sys.float_info.epsilon)
            weighted_sum = (weights * nears[:, 1]).sum()
            ret[i, j] = weighted_sum / weights.sum()
    return ret

In [0]:
import sys
from multiprocessing import Process, Manager

n_procs = 4
procs = []
manager = Manager()
dpr_preci = manager.list([None] * n_procs)

# split train into n_procs arrays
n_imgs = test_submit.shape[0]
splitted = np.split(test_submit, np.arange((n_imgs + n_procs - 1) // n_procs,
                                     n_imgs, n_imgs // n_procs))

def proc_func(proc_id):
    part = splitted[proc_id]
    arr = np.empty_like(part[:, :, :, 14])  # shape=(-1, 40, 40)
    for i in range(part.shape[0]):
        arr[i, :, :] = compen_ll(part[i, :, :, 14],
                                 part[i, :, :, 10:12],
                                 part[i, :, :, 12:14])
        if (i + 1) % 200 == 0:
            print(proc_id, i + 1)
    dpr_preci[proc_id] = arr

for proc_id in range(n_procs):
    proc = Process(target=proc_func, args=(proc_id, ))
    proc.start()
    procs.append(proc)

for proc in procs:
    proc.join()

dpr_preci = np.concatenate(dpr_preci)
dpr_preci = dpr_preci.reshape(-1, 40, 40, 1)
# dpr_preci = dpr_preci.astype(np.float32)

In [0]:
# 훈련에서 사용하는 0~8번 채널, ladtype 만을 뽑아서 X_test에 저장하기
test_submit = np.load('/content/drive/My Drive/2020 Kaggle Study/data/test.npy')
test_submit = test_submit[:,:,:,:10]
del test_submit
gc.collect()
pred = model.predict(test_submit)

## submission 만들기

In [0]:
submission = pd.read_csv('/content/drive/My Drive/2020 Kaggle Study/data/sample_submission.csv')

In [0]:
submission.iloc[:,1:] = pred.reshape(-1, 1600)

In [0]:
submission.to_csv('Dacon_landtype.csv', index = False)

In [0]:
# 제출파일 구글 드라이브에 올리기
!cp Dacon_landtype.csv '/content/drive/My Drive/2020 Kaggle Study/submission'